# Current work cell

In [0]:
!pip install mysql.connector

In [0]:
import mysql.connector
from mysql.connector import Error
import requests
import pandas as pd
from math import cos, asin, sqrt
from datetime import datetime

textfile = open("drive/My Drive/sqlfile.txt", "a")

connection = mysql.connector.connect(host='lambdalabs22sfmta.crqoxlwn3q6h.us-east-2.rds.amazonaws.com',
                                         database='lambdalabs22',
                                         user='admin',
                                         password='13YqLw2Gs3umAVNG9hVe')

def distance(lat1, lon1, lat2, lon2, stopId):
    p = 0.017453292519943295
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p)*cos(lat2*p) * (1-cos((lon2-lon1)*p)) / 2
    return [12742 * asin(sqrt(a)), stopId]

def closest(data, v):
    return min(data, key=lambda p: distance(v['lat'],v['lon'],p['stop_lat'],p['stop_lon'], p['stopId']))
    return c_temp[0], c_temp[1]


stops = pd.read_csv('https://raw.githubusercontent.com/Lambda-School-Labs/sfmta-data-analysis-ds/master/datasets/route_info.csv')

rid_list = stops['route_id'].unique()


thingtowrite = ""

for rid in rid_list[1:-1]:
    generic_dict = {'%_I_%':[], '%_O_%':[]}

    for i, row in stops[stops['route_id'] == rid].iterrows():
        if row['dir'] == 'Outbound':
            #generic_dict[outbound].append -> {'lat':e,'lon':stops['lon'][i]}
            # Need stop id not route id
            generic_dict['%_O_%'].append({'stop_lat':row['lat'],'stop_lon':row['lon'], 'stopId': row['stopId']})
        else:
            generic_dict['%_I_%'].append({'stop_lat':row['lat'],'stop_lon':row['lon'], 'stopId': row['stopId']})


    # for key, value_dict in enumerate(generic_dict):
    for key, value_dict in generic_dict.items():

      # Consider adding rid & dir variables if we do all routes
        cur = connection.cursor()
        sql_query = ('''SELECT id, datetime, rid, vid, secs, kph, head, lat, lon, dir
                        FROM historic_location
                        WHERE rid = %s
                        AND
                        dir LIKE %s;
                        ''')
        cur.execute(sql_query, (rid, key))

        results = cur.fetchall()

        df_key = pd.DataFrame(results,
                            columns=['id', 'datetime', 'rid', 'vid', 'secs',
                                    'kph', 'head', 'lat', 'lon', 'dir'])
        new_values = []

        for index, row in df_key.iterrows():
            v = {'lat': float(row['lat']), 'lon': float(row['lon'])}
            new_values.append(closest(value_dict, v))


        # Make DF of new_values list
        temp_df = pd.DataFrame(new_values)
        temp_df.head()

        # concatenate queried DF and temp_df list
        results = pd.concat([df_key, temp_df], axis=1)

        sql_values = []
        for index, row in results.iterrows():
          thingtowrite += f"INSERT INTO historic_location_stops (id,datetime,rid,vid,secs,kph,head,lat,lon,dir,stop_lat,stop_lon,stop_id) VALUES ({row['id']}, '{row['datetime'].strftime('%Y-%m-%d %H:%M:%S')}', '{str(row['rid'])}', {row['vid']},{row['secs']}, {row['kph']},{row['head']}, {row['lat']}, {row['lon']}, '{str(row['dir'])}',{row['stop_lat']}, {row['stop_lon']}, {row['stopId']});\n"

textfile.write(thingtowrite)
textfile.close()


In [0]:
textfile = open("drive/My Drive/sqlfile.txt", "a")

In [0]:
from google.colab import files
files.download('sql.txt')

In [0]:
from google.colab import drive
drive.mount('/content/drive')

# This code is the prior version of the large cell above. Perhaps it's useful to study.

In [0]:
import mysql.connector
from mysql.connector import Error
import pandas as pd
from matplotlib import pyplot as plt

connection = mysql.connector.connect(host='lambdalabs22sfmta.crqoxlwn3q6h.us-east-2.rds.amazonaws.com',
                                         database='lambdalabs22',
                                         user='admin',
                                         password='13YqLw2Gs3umAVNG9hVe')

# Importing stops data
stops = pd.read_csv('https://raw.githubusercontent.com/Lambda-School-Labs/sfmta-data-analysis-ds/master/datasets/route_info.csv')

outbound = stops[stops['dir'] == 'Outbound']
inbound = stops[stops['dir'] == 'Inbound']

# Variables that may be user input
rid = '1'
datetime = ''
i_or_o = '%_I_%'

# Filter Stops DF for 1 route
# This should be saved / responsive to user input
inbound = inbound[inbound['route_id'] == rid]
outbound = outbound[outbound['route_id'] == rid]

## list of route-inbound & route-outbound from stop data -> array of lat/lons
inbound_array = inbound[['lat', 'lon', 'stopId']]
outbound_array = outbound[['lat', 'lon', 'stopId']]

# Stops Array -> Dict
inbound_dict = inbound_array.to_dict()
outbound_dict = outbound_array.to_dict()


# Get dict of 'lat', 'long', 'stopId'
list_dict = []

for key, values in inbound_dict['lon'].items():
  list_dict.append({'stop_lat': inbound_dict['lat'][key], 
                    'stop_lon': values, 
                    'stopId': inbound_dict['stopId'][key]})
  
## database call -> SELECT * FROM historic_location WHERE rid=route & dir CONTAINS __I (inbound) __O (outbound) -> dataframe
cur = connection.cursor()
sql_query = ('''SELECT id, datetime, rid, vid, secs, kph, head, lat, lon, dir
                FROM historic_location
                WHERE rid = %s
                AND
                dir LIKE %s
                LIMIT 1000000;
                ''')

cur.execute(sql_query, (rid, i_or_o))

# Create DF of pertinent columns
results = cur.fetchall()
df_inbound = pd.DataFrame(results,
                          columns=['id', 'datetime', 'rid', 'vid', 'secs',
                                   'kph', 'head', 'lat', 'lon', 'dir'])

## for loop index, row df.iterrows -> lat = row['lat'] & lon = row['lon]
## -> pass into the function from starter code
from math import cos, asin, sqrt

# Math functions to calculate closest stop
def distance(lat1, lon1, lat2, lon2, stopId):
    p = 0.017453292519943295
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p)*cos(lat2*p) * (1-cos((lon2-lon1)*p)) / 2
    return [12742 * asin(sqrt(a)), stopId]

def closest(data, v):
    return min(data, key=lambda p: distance(v['lat'],v['lon'],p['stop_lat'],p['stop_lon'], p['stopId']))
    return c_temp[0], c_temp[1]

# Add lat, long from Stops DF and StopId in calculated order to a list
new_values = []
## for loop index, row df.enumerate -> lat = row['lat'] & lon = row['lon]
## -> pass into the function from starter code
for index, row in df_inbound.iterrows():
    # lat = float(row['lat'])
    # lon = float(row['lon'])
    lat = row['lat']
    lon = row['lon']
    v = {'lat': float(lat), 'lon': float(lon)}
    new_values.append(closest(list_dict, v))


# Make DF of new_values list
temp_df = pd.DataFrame(new_values)
temp_df.head()

# concatenate queried DF and temp_df list
results = pd.concat([df_inbound, temp_df], axis=1)

## This is beyond the large cell above

# Sort DF by stopId and datetime
results_sorted = results.sort_values(by=['stopId', 'datetime'])

# Stretch: Isolate data for the requested route

# Isolate data for the requested date
cali1_3_29 = results_sorted.loc[(results_sorted['datetime'].dt.month==3) & (results_sorted['datetime'].dt.day==29)]

# Acquire sorted list of bus stops on requested route
# Possibly must exclude first and last stop
stopIds = [13555, 13548, 13546, 13844, 13842, 13840, 13838, 13836, 13834, 13887,
           1382, 13830, 13827, 13825, 13823, 13846, 13853, 13857, 13876, 13893,
           13848, 13859, 13885, 16489, 16292, 16320, 16292, 16306, 16310, 14905,
           14016, 14031, 14026, 14022, 14019, 14023, 14020, 14030, 14024, 14027,
           14029, 14081, 14021, 14025, 14028, 14017]

# Filter DF for stopIds list
cali1_final =  cali1_3_29[cali1_3_29['stopId'].isin(stopIds)]

# Sort DF by Stops order & datetime
sorterIndex = dict(zip(stopIds,range(len(stopIds))))
cali1_final['stopId_sorted'] = cali1_final['stopId'].map(sorterIndex)

cali1_final.sort_values(['stopId_sorted', 'datetime'],ascending = [True, True], inplace = True)
cali1_final.drop('stopId_sorted', 1, inplace = True)

# Create new column containing time delta between busses
cali1_final['difference'] = cali1_final['datetime'] - cali1_final['datetime'].shift()

# Drop first row of DF due to NaN value (maybe not required)
cali1_final = cali1_final.drop(197990)

# Create a column 'difference' in minutes
cali1_final['difference_min'] = cali1_final['difference'].astype('timedelta64[m]')

# Note: This step is currently unused and issues with graphing.
# Create a column of 'difference_min' binned to visualize time between vehicles
# cali1_final['difference_binned'] = pd.cut(cali1_final['difference_min'], bins=[0, 5, 10, 15, 20, 25, 30])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.6/dist-packa

In [0]:
# Very rough notes

# Bunching and gapping notes are in "B&G visz" notebook

# "Pipeline":
# "API" data is merged with Stops data at certain intervals (each day/week)
# User requests date
# Date is input into SQL query?
# Data is filtered by stopId (correct order) and datetime (stored?)
# (consider) reducing one day into one row for ease of analysis / presentation
# Data (datetime, vid, rid, lat, long, stopid) is sent to front end?
# Graph is displayed on front end

In [0]:
# Maybe make a boxplot
# min / max, median is average
# What bus was soonest / slowest
# Get an idea for what these look like at different times of day

# Another chart:
# line graph , one stop.
# one bus on the stops above
# look at time between  stops (outliers)

# Does datetime have a subtract function?

# Hist / heatmap of time between vehicles

# visz on map?
# animate? This is stretch

# Compute the calculations for the viszs in advance, so they render smoothly
# May mean that certain recent dates aren't selectable.